<font color='green' size='7'>**Домашняя работа**</font>  
<font color='#11a642' size='6'>**Тема: Машинное обучение. Алгоритм Дерево решений**</font>  
<font size='5'>**Задание:**</font>  

Домашняя работа основана на данных активного конкурса на 11-07-2024 https://www.kaggle.com/competitions/playground-series-s4e7.

Описание данных https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data/discussion/516324

1. Загрузите данные c конкурса https://www.kaggle.com/competitions/playground-series-s4e7.
2. Проведите развелочный анализ данных (EDA):  
3. Построение модели дерево решений
4. Опционально. Загрузите решение на kaggle


P.S. Если ваш код будет падать по памяти, создайте случайную подвыборку и обучайте модель на случайно подвыборке. При создании подвыборки учтите распределение target и других признаков





# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [4]:
from google.colab import drive
import json
import zipfile

import pandas as pd

# <font color='#11a642' size='6'> **Загрузка данных**

In [5]:
def load_dataset(from_kaggle=False) -> pd.DataFrame:
  '''
  Функция скачивает данные с сайта kaggle, если установлен from_kaggle=True,
  инчае архив считывается по ссылке с моегого гугл диска (такой способ удобен тем, у кого нет доступа к kaggle)
  '''
  if from_kaggle:
    # запросит разрешение к гугл диску, необходимо дать это разрешение
    drive.mount('/content/drive')
    # установим kaggle
    !pip install kaggle -q
    !mkdir ~/.kaggle
    # копируем kaggle.json (предварительно, необходимо сгенерить токен на
    # сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
    !cp "/content/drive/MyDrive/Colab Notebooks/config/kaggle.json" ~/.kaggle/
    !kaggle competitions download -c playground-series-s4e7
  else:
    !gdown 1HG4oNC-EfEK7DsXDfKKfb5BtgzevyZZO
  # распаковка архива
  zip_ref = zipfile.ZipFile('playground-series-s4e7.zip', 'r')
  zip_ref.extractall()
  zip_ref.close()
  df_train = pd.read_csv('train.csv')
  df_test = pd.read_csv('test.csv')
  df_sample_submission = pd.read_csv('sample_submission.csv')
  return df_train, df_test, df_sample_submission

In [6]:
df_train, df_test, df_sample_submission = load_dataset(from_kaggle=False)

Downloading...
From (original): https://drive.google.com/uc?id=1HG4oNC-EfEK7DsXDfKKfb5BtgzevyZZO
From (redirected): https://drive.google.com/uc?id=1HG4oNC-EfEK7DsXDfKKfb5BtgzevyZZO&confirm=t&uuid=5f945e7d-163d-41d9-9846-422d99a8d010
To: /content/playground-series-s4e7.zip
100% 259M/259M [00:07<00:00, 35.2MB/s]


In [7]:
# это данные даны для примера, в таком виде необходимо сделать выборку и загрузить на kaggle
#  (для тех, кто хочет увидеть себя в лидерборде и оценить на сколько ваше решение лучше/хуже остальных)
df_sample_submission.head()

,id,Response
0,11504798,0.5
1,11504799,0.5
2,11504800,0.5
3,11504801,0.5
4,11504802,0.5


In [8]:
# это тестовые данные, для которых необходимо предсказать target, то есть применить модель,
#  которую вы разработаете на данных df_train
df_test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [9]:
# на этих данных дальше необходимо разработать модель
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


## <font color='#11a642' size='5'> Определим роли для независимых признаков

In [10]:
id = 'id'
target = 'Response'

In [11]:
# на основании описание данных
#  https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data/discussion/516324

features_cat_from_description = ['Region_Code', 'Vehicle_Age', 'Policy_Sales_Channel']

In [12]:
features_num = [feature for feature in df_train.select_dtypes(include='number').columns.to_list()
               if feature not in [id, target, *features_cat_from_description]]
features_num

['Age', 'Driving_License', 'Previously_Insured', 'Annual_Premium', 'Vintage']

In [13]:
features_cat = [feature for feature in df_train.columns.to_list() if feature
                not in [id, target, *features_num]]
features_cat

['Gender',
 'Region_Code',
 'Vehicle_Age',
 'Vehicle_Damage',
 'Policy_Sales_Channel']

In [14]:
features_cat_bin = ['Gender', ]

# <font color='#11a642' size='6'> **Разведочный анализ данных**

## <font color='#11a642' size='5'> Оцените размер вашего датасета


1.   Сколько ГБ ваш датасет
2.   Сколько строк и столбцов



In [15]:
# Сколько ГБ ваш датасет
# Проверка размера файла в мегабайтах
import sys

memory_usage_gb = df_train.memory_usage(deep=True).sum() / (1024 * 1024 * 1024)  # размер в ГБ
print(f"Размер DataFrame в памяти: {memory_usage_gb:.2f} ГБ")

Размер DataFrame в памяти: 2.77 ГБ


In [16]:
# Сколько строк и столбцов
num_rows, num_columns = df_train.shape

print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_columns}")

Количество строк: 11504798
Количество столбцов: 12


## <font color='#11a642' size='5'> Анализ категориальных признаков


In [17]:
for column in df_train.select_dtypes(include=['object', 'category']):
    print(f"{column}: {df_train[column].nunique()} уникальных значений")

Gender: 2 уникальных значений
Vehicle_Age: 3 уникальных значений
Vehicle_Damage: 2 уникальных значений


In [18]:
for column in df_train.select_dtypes(include=['object', 'category']):
    print(df_train[column].value_counts())

Gender
Male      6228134
Female    5276664
Name: count, dtype: int64
Vehicle_Age
1-2 Year     5982678
< 1 Year     5044145
> 2 Years     477975
Name: count, dtype: int64
Vehicle_Damage
Yes    5783229
No     5721569
Name: count, dtype: int64


## <font color='#11a642' size='5'> Анализ пропусков


In [19]:
missing_values = df_train.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


# <font color='#11a642' size='6'> **Подготовка данных к обучению**



1.   Обработка категориальных переменных
2.   Обработка пропусков
3.   Стоит ли делать нормализацию данных?



In [20]:
df_encoded = pd.get_dummies(df_train, drop_first=True)

Нормализация данных (приведение всех признаков к единому масштабу) не является необходимой при использовании моделей на основе деревьев решений, таких как Decision Tree, Random Forest или Gradient Boosting.

Деревья решений работают с разделением данных на основе предикатов (условий типа <= или >), поэтому масштаб признаков не влияет на их работу. Модель оценивает различные разбиения для каждого признака независимо от его масштаба. Поэтому нормализация данных не требуется перед использованием дерева решений.

# <font color='#11a642' size='6'> **Построение модели дерево решений**

## <font color='#11a642' size='5'> Построение максимального дерева без ограничений на гиперпараметры


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X = df_encoded.drop(['Response', 'id'], axis=1)
y = df_encoded.Response

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Построение максимального дерева без ограничений
max_tree = DecisionTreeClassifier(random_state=42)
max_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

### <font color='#11a642' size='4'> Оценка качества  максимального дерева без ограничений на гиперпараметры


In [22]:
# Оценка качества максимального дерева
y_pred = max_tree.predict(X_test)
accuracy_max_tree = accuracy_score(y_test, y_pred)
print(f"Точность максимального дерева без ограничений: {accuracy_max_tree:.4f}")

Точность максимального дерева без ограничений: 0.8321


### <font color='#11a642' size='4'> Ваши выводы


Точность: Значение точности 0.8321 (или 83.21%) указывает на то, что дерево решений без ограничений на гиперпараметры правильно классифицировало примерно 83.21% случаев в тестовом наборе данных.

Переобучение: Так как дерево решений строится без ограничений, оно может подгонять данные под тренировочный набор, что иногда приводит к переобучению. Это означает, что дерево может хорошо справляться с обучением на тренировочных данных, но его способность к обобщению на новые данные может быть снижена.

## <font color='#11a642' size='5'> Построение дерева с поиском наилучших гиперапарметров


In [23]:
from sklearn.model_selection import GridSearchCV

# Определение параметров для Grid Search
param_grid = {
    'max_depth': [3, 5, 7]
}

# Поиск наилучших гиперпараметров с использованием Grid Search
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7]}, verbose=1)

### <font color='#11a642' size='4'> Оценка качества  дерева с оптимальными гиперпараметрами


In [24]:
best_tree = grid_search.best_estimator_
y_pred_best = best_tree.predict(X_test)
accuracy_best_tree = accuracy_score(y_test, y_pred_best)
print(f"Точность дерева с оптимальными гиперпараметрами: {accuracy_best_tree:.4f}")

Точность дерева с оптимальными гиперпараметрами: 0.8770


### <font color='#11a642' size='4'> Важность признаков для дерева с оптимальными гиперпараметрами


In [27]:
import pandas as pd
import numpy as np

feature_importances = best_tree.feature_importances_
features = X.columns  # Используем имена столбцов DataFrame
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Важность признаков:")
print(importance_df)

Важность признаков:
                  Feature  Importance
10     Vehicle_Damage_Yes    0.783485
0                     Age    0.203182
5    Policy_Sales_Channel    0.010245
3      Previously_Insured    0.003088
1         Driving_License    0.000000
2             Region_Code    0.000000
4          Annual_Premium    0.000000
6                 Vintage    0.000000
7             Gender_Male    0.000000
8    Vehicle_Age_< 1 Year    0.000000
9   Vehicle_Age_> 2 Years    0.000000


### <font color='#11a642' size='4'> Ваши выводы


Интерпретация результата:

Точность: Значение точности 0.8770 (или 87.70%) указывает на то, что дерево решений, настроенное с помощью оптимальных гиперпараметров, правильно классифицировало 87.70% случаев в тестовом наборе данных. Это улучшение по сравнению с моделью без ограничений гиперпараметров (83.21%).

Улучшение модели: Повышение точности указывает на то, что настройка гиперпараметров значительно улучшила производительность модели. Это говорит о том, что оптимизация гиперпараметров позволяет модели лучше обобщать данные и достигать более высокой точности на новых данных.

# <font color='#11a642' size='6'> **Опционально, для тех, кто хочет сравнить свое решение с другими решениями на kaggle. Применение алгоритма к тестовой выборке df_test и сабмит решения на kaggle**

In [ ]:
# ваш код

In [ ]:
# # загрузка данных на kaggle (нужен доступ на kaggle)
# !kaggle competitions submit -c playground-series-s4e7 -f /content/submissions.csv -m "1 submit"